# Lab 3: Digital imagery and image processing

**Purpose:** The purpose of this lab is to demonstrate concepts of digital image processing.  You will be introduced to methods for image smoothing, sharpening, edge detection, morphological processing, texture analysis, resampling and reprojection.  At the completion of the lab, you will be able to identify image processing operators that may be useful in extracting information of interest for your image analyses.

In [ ]:
# import ee api and geemap package
import ee
import geemap
from pprint import pprint

In [ ]:
# try to initalize an ee session
# if not authenticated then run auth workflow and initialize
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Digital image visualization

You've learned about how an image stores pixel data in each band as DNs and how the pixels are organized spatially.  When you add an image to the map, Earth Engine handles the spatial display for you by recognizing the projection and putting all the pixels in the right place.  However, you must specify how to stretch the DNs to make an 8-bit display image (e.g. the min and max visualization parameters).  Specifying min and max applies (where $DN'$ is the displayed value):

$DN' = (DN - min) * 255 / (max - min)$

This visualization process is linear, we can apply transformations on the displayed values to highlight specific features of the image or values.

In [ ]:
# load in a Landsat 8 image directly 
# this is the image we will be using for processing
image = (
    ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_044034_20140318')
    .select("B[1-7]")
)

Some of the image processing we are going to explore requires us to "force" Earth Engine to process at a specific projection and scale so we are going to set the image projection to a variable for later use.

In [ ]:
# extract out the projection information
proj = image.projection()

### Gamma correction

The Gamma correction is a nonlinear operation used to scale the DN values for visualization. The gamma correction can be applied mathematically ($DN' = DN^\gamma$), however, we can apply it simply providing the `gamma` keyword for visualization:

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

# Display gamma stretches of the input image.
Map.addLayer(image.visualize(bands=["B7","B5","B3"], min=0, max=5500, gamma=0.5), {}, 'gamma = 0.5');
Map.addLayer(image.visualize(bands=["B7","B5","B3"], min=0, max=5500, gamma=1.5), {}, 'gamma = 1.5');

Map.addLayerControl()

Map

Note that gamma is supplied as an argument to `image.visualize()` so that you can click on the map to see the difference in pixel values (try it!).  It's possible to specify gamma, min and max to achieve other unique visualizations.


### Histogram equalization

Histogram equalization is a method in image processing of contrast adjustment using the image's histogram.

To apply a histogram equalization stretch, use the `sldStyle()` method:


In [ ]:
# Define a RasterSymbolizer element with '_enhance_' for a placeholder.
histogram_sld = """
  <RasterSymbolizer>
    <ContrastEnhancement><Histogram/></ContrastEnhancement>
    <ChannelSelection>
      <RedChannel>
        <SourceChannelName>B7</SourceChannelName>
      </RedChannel>
      <GreenChannel>
        <SourceChannelName>B5</SourceChannelName>
      </GreenChannel>
      <BlueChannel>
        <SourceChannelName>B3</SourceChannelName>
      </BlueChannel>
    </ChannelSelection>
  </RasterSymbolizer>
"""


In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

# Display visualization stretches of the input image.
Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(image.sldStyle(histogram_sld), {}, 'Equalized');


Map.addLayerControl()

Map

## Band math

Band math can be performed using operators like `add()` and `subtract()`, but for complex computations with more than a couple of terms, the `expression()` function provides a good alternative.

### Normalized Difference Vegetation Index

As a simple example for using band math is calculating the Normalized Difference Vegetation Index (NDVI) using Landsat imagery, where `add()`, `subtract()`, and `divide()` operators are used:

$NDVI = \frac{NIR-Red}{NIR + Red}$

In [ ]:
nir = image.select("B5")
red = image.select("B4")

ndvi = nir.subtract(red).divide(nir.add(red))

In [ ]:
def ndvi_exp(image):
    ndvi_expr = image.expression('(nir-red)/(nir + red)', {
      "nir": image.select("B5"),
      "red": image.select("B4")
      })
    return ndvi_exp  


In [ ]:
# Display original and NDVI images.
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original')
Map.addLayer(ndvi, {"min":0,"max":1}, 'NDVI');


Map.addLayerControl()

Map

For the complete list of mathematical operators handling basic arithmetic, trigonometry, exponentiation, rounding, casting, bitwise operations and more, see the [API documentation](https://developers.google.com/earth-engine/apidocs).

### Expression example

To implement more complex mathematical expressions, consider using `image.expression()`, which parses a text representation of a math operation. The following example uses `expression()` to compute the [Enhanced Vegetation Index (EVI)](https://en.wikipedia.org/wiki/Enhanced_vegetation_index):

In [ ]:
evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': image.select('B5'),
        'RED': image.select('B4'),
        'BLUE': image.select('B2')
})

In [ ]:
# Display original and NDVI images.
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original')
Map.addLayer(ndvi, {"min":0,"max":1}, 'NDVI')
Map.addLayer(evi, {"min":0,"max":2}, 'EVI')


Map.addLayerControl()

Map

The first argument to `expression()` is the textual representation of the math operation, the second argument is a dictionary where the keys are variable names used in the expression and the values are the image bands to which the variables should be mapped. Bands in the image may be referred to as `b("band name")` or `b(index)`, for example `b(0)`, instead of providing the dictionary. Bands can be defined from images other than the input when using the band map dictionary

## Zonal statistics

To get statistics of pixel values in a region of an ee.Image, use `image.reduceRegion()`. This reduces all the pixels in the region(s) to a statistic or other compact representation of the pixel data in the region (e.g. histogram). The region is represented as a Geometry, which might be a polygon, containing many pixels, or it might be a single point, in which case there will only be one pixel in the region.



In [ ]:
# create a combined reducer that will calculate mean and standard deviation 
my_reducer = ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True)

# calculate mean and stdDev over entire image
image_stats = image.reduceRegion(
    reducer = my_reducer,
    geometry = image.geometry(),
    scale = 120,
    maxPixels = 1e10,
    bestEffort = False
)

Note here that the reducers are combined as it is more efficient to combine reducers if you need multiple statistics (e.g. mean and standard deviation) from a single input (e.g. an image region). ([Combining reducers best practice](https://developers.google.com/earth-engine/guides/best_practices#combine-reducers))

In [ ]:
# print the statistics
image_stats.getInfo()

In [ ]:
# convert the dictionary to an image
# band names will be the keys
# bands will be constant values
statistics_image = image_stats.toImage()

In [ ]:
# extract out the mean and standard deviation bands
# to two seperate images
mean_image = statistics_image.select("B.*_mean")
stdDev_image = statistics_image.select("B.*_stdDev")

# calculate z-score
zscore = image.select("B.*").subtract(mean_image).divide(stdDev_image)

In [ ]:
# Display original and scaled images.
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original')
Map.addLayer(zscore, {"bands": ["B7","B5","B3"],"min":-2.5,"max":2.5}, 'Std. Deviation Stretch')

Map.addLayerControl()

Map

## Neighborhood operations


### Linear filters

In the present context, linear filtering (or [convolution](http://www.dspguide.com/ch24/1.htm) refers to a linear combination of pixel values in a neighborhood.  The neighborhood is specified by a [kernel](https://en.wikipedia.org/wiki/Kernel_(image_processing)), where the weights of the kernel determine the coefficients in the linear combination.  (For this lab, the terms kernel and filter are interchangeable.)  Filtering an image can be useful for extracting image information at different spatial frequencies.  For this reason, smoothing filters are called low-pass filters (they let low-frequency data pass through) and edge detection filters are called high-pass filters.  To implement filtering in Earth Engine use `image.convolve()` with an `ee.Kernel` for the argument.


#### Smoothing. 

Smoothing means to convolve an image with a smoothing kernel.  A simple smoothing filter is a square kernel with uniform weights that sum to one.  Convolving with this kernel sets each pixel to the mean of its neighborhood.  Print a square kernel with uniform weights (this is sometimes called a "pillbox" or "boxcar" filter):


In [ ]:
# Define a uniform kernel and print to see its weights.
uniform_kernel = ee.Kernel.square(2)

pprint(uniform_kernel.getInfo());


In [ ]:
# Filter the image by convolving with the smoothing filter.
smoothed = image.convolve(uniform_kernel)

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(smoothed, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Smoothed');

Map.addLayerControl()

Map

To make the image even more smooth, try increasing the size of the neighborhood by increasing the pixel radius.

A Gaussian kernel can also be used for smoothing.  Think of filtering with a Gaussian kernel as computing the weighted average in each pixel's neighborhood.  For example:

In [ ]:
# define a gaussian kernel
gaussian_kernel = ee.Kernel.gaussian(2)

pprint(gaussian_kernel.getInfo(),width = 500)

In [ ]:
# apply the gaussian kernel on the image
gaussian = image.convolve(gaussian_kernel)

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(gaussian, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Gaussian');

Map.addLayerControl()

Map

#### Edge Detection

Convolving with an edge-detection kernel is used to find rapid changes in DNs that usually signify edges of objects represented in the image data. 

A classic edge detection kernel is the [Sobel](https://en.wikipedia.org/wiki/Sobel_operator) kernel.  Investigate the kernel weights and the image that results from convolving with the Sobel kernel:


In [ ]:
# Define a Sobel filter.
x_sobel_kernel = ee.Kernel.sobel()
y_sobel_kernel = x_sobel_kernel.rotate(1)

In [ ]:
# Print the kernel to see its weights.
pprint(x_sobel_kernel.getInfo())
pprint(y_sobel_kernel.getInfo())


In [ ]:
x_edges = (
    image
    .convolve(x_sobel_kernel)
)

y_edges = (
    image
    .convolve(y_sobel_kernel)
)

edges = (
    x_edges.pow(2).add(y_edges.pow(2)).sqrt()
    .reproject(proj, None, proj.nominalScale()) # force processing at native projection for visualization
)

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(edges, {"min": 0, "max": 2000,}, 'Edges');

Map.addLayerControl()

Map

(Ignore the `reproject()` call for now.  It is explained in section 7.)

Other edge detection kernels include the [Laplacian](https://en.wikipedia.org/wiki/Discrete_Laplace_operator), [Prewitt](https://en.wikipedia.org/wiki/Prewitt_operator) and [Roberts](https://en.wikipedia.org/wiki/Roberts_cross) kernels.  [Learn more about additional edge detection methods in Earth Engine](https://developers.google.com/earth-engine/image_edges)

#### Sharpening.  

Image sharpening, or edge enhancement, is related to the idea of the image second derivative.  Specifically, mimic the perception of Mach bands in human optical response by adding the image to its second derivative.

One implementation of this idea is to convolve an image with a Laplacian-of-a-Gaussian or [Difference-of-Gaussians](https://en.wikipedia.org/wiki/Difference_of_Gaussians) filter (see [Schowengerdt 2007](http://www.sciencedirect.com/science/book/9780123694072) for details), then add that to the input image:


In [ ]:
# Define a "fat" Gaussian kernel.
fat = ee.Kernel.gaussian(
  radius= 3,
  sigma= 3,
  magnitude= -1,
  units= 'meters'
)

In [ ]:
# Define a "skinny" Gaussian kernel.
skinny = ee.Kernel.gaussian(
  radius= 3,
  sigma= 0.5,
  units= 'meters'
)

In [ ]:
# Compute a difference-of-Gaussians (DOG) kernel.
dog = fat.add(skinny)


In [ ]:
# Add the DoG filtered image to the original image.
sharpened = (
    image.add(image.convolve(dog))
    .reproject(proj, None, proj.nominalScale()) # force processing at native projection for visualization
)

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

# Display gamma stretches of the input image.
Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(sharpened, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Sharpened');

Map.addLayerControl()

Map

Related concepts include [*spectral inversion*](http://www.dspguide.com/ch14/5.htm) from digital signal processing and unsharp masking ([Burger and Burge 2008](http://imagingbook.com/)).


### Non-linear filtering

The previous convolution examples can all be implemented as linear combinations of pixel values in a neighborhood (edge detection sometimes needs a couple extra steps, but nevermind that).  Non-linear functions applied to a neighborhood are also useful.  Implement these functions in Earth Engine with the `reduceNeighborhood()` method on images. 

#### Median

A median filter can be useful for denoising images.  Specifically, suppose that random pixels in your image are saturated by anomalously high or low values that result from some noise process.  Filtering the image with a mean filter (as in the "Smoothing" section) would result in pixel values getting polluted by noisy data.  To avoid that, smooth the image with a median filter (reusing the 5x5 uniform kernel from above):


In [ ]:
median = image.reduceNeighborhood(
  reducer= ee.Reducer.median(), 
  kernel= uniform_kernel
).rename(image.bandNames())


In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

# Display gamma stretches of the input image.
Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(median, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Median');

Map.addLayerControl()

Map

#### Mode

For categorical maps, methods such as median and mean make little sense for aggregating nominal data.  In these cases, use neighborhood mode to get the most frequently occurring value.

For demonstration purposes, we will load in another dataset that represents land cover and apply the mode filter on:


In [ ]:
# load in a clssified land cover image from the NLCD collection
landcover = (
    ee.ImageCollection("USGS/NLCD_RELEASES/2016_REL")
    .first()
    .select(['landcover'])  # Select the classification band of interest.
)

In [ ]:
# Smooth with a mode filter.
mode_filtered = landcover.focal_mode(radius=2);

# this is the equivalent calling reduceNeigborhood with 
# a mode reducer and 5x5 uniform kernel
# image.reduceNeighborhood(
#     reducer = ee.Reducer.mode(),
#     ...
# )

In [ ]:
# copy properties from the original landcover image so it displays correctly
mode_filtered = ee.Image(mode_filtered.copyProperties(landcover))

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 7)

Map.addLayer(landcover, {}, 'Original Landcover');
Map.addLayer(mode_filtered, {}, 'Landcover mode');

Map.addLayerControl()


Map

## Image Segementation

In [ ]:
# alternate approach for calculating a normalized difference
mndwi = image.normalizedDifference(["B3","B6"])

In [ ]:
threshold = 0.1 # define a threshold to segment the image
water = mndwi.gt(threshold)

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(mndwi, {"min": -0.5, "max": 1,}, 'MNDWI');
Map.addLayer(water, {"min": 0, "max": 1, "palette":"black,lightblue"}, 'Water');

Map.addLayerControl()


Map

## Morphological operations

The idea of morphology is tied to the concept of objects in images.  For example, suppose the patches of 1's in the water image from the previous section represent patches of water.

###Dilation (max). 

If the patches underestimate the actual distribution of water, or contain "holes", a max filter can be applied to [dilate](https://en.wikipedia.org/wiki/Dilation_(morphology)) the areas of water:


In [ ]:
# Dilate by takaing the max in each 5x5 neighborhood.
max = water.reduceNeighborhood(
  reducer= ee.Reducer.max(), 
  kernel= uniform_kernel
)

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(water, {"min": 0, "max": 1, "palette":"black,lightblue"}, 'Water');
Map.addLayer(max, {"min": 0, "max": 1, "palette":"black,lightblue"}, 'Dilation');

Map.addLayerControl()


Map

### Erosion (min).  

The opposite of dilation is [erosion](https://en.wikipedia.org/wiki/Erosion_(morphology), for decreasing the size of the patches:

In [ ]:
# Erode by takaing the min in each 5x5 neighborhood
min = water.reduceNeighborhood(
  reducer= ee.Reducer.min(), 
  kernel= uniform_kernel
)

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(water, {"min": 0, "max": 1, "palette":"black,lightblue"}, 'Water');
Map.addLayer(min, {"min": 0, "max": 1, "palette":"black,lightblue"}, 'Erosion');

Map.addLayerControl()


Map

### Opening

To "open" possible "holes" in the patches, perform an erosion followed by a dilation.  This process is called [opening](https://en.wikipedia.org/wiki/Opening_(morphology)).  Try that by performing a dilation of the eroded image:


In [ ]:
# Perform an opening by dilating the eroded image.
opened = min.reduceNeighborhood(
  reducer= ee.Reducer.max(), 
  kernel= uniform_kernel
)


In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(water, {"min": 0, "max": 1, "palette":"black,lightblue"}, 'Water');
Map.addLayer(opened, {"min": 0, "max": 1, "palette":"black,lightblue"}, 'Opened');

Map.addLayerControl()


Map

### Closing

The opposite of opening is [closing](https://en.wikipedia.org/wiki/Closing_(morphology)), or dilation followed by a erosion.  Use this to "close" possible "holes" in the input patches:


In [ ]:
# Perform a closing by eroding the dilated image.
closed = max.reduceNeighborhood(
  reducer= ee.Reducer.min(), 
  kernel= uniform_kernel
)


In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(water, {"min": 0, "max": 1, "palette":"black,lightblue"}, 'Water');
Map.addLayer(closed, {"min": 0, "max": 1, "palette":"black,lightblue"}, 'Closed');

Map.addLayerControl()


Map

Examine the difference between each morphological operation and the water input.  Tune these morphological operators by adjusting the size and shape of the kernel (also called a [*structuring element*](https://en.wikipedia.org/wiki/Structuring_element) in this context, because of its effect on the shape of the result), or applying the operations repetively.

## Compositing

Compositing is the process of taking multiple images and making a single representative image using some statistical reduction. There are multiple ways to achieve composites and an active research area. Really the composite apporach you use depends on the data and application.

### Mean vs Median Composite

Mean/Medain compositing is very common and this illustrates the general workflow to create such a composite.

In [ ]:
# load in an image collection and filter by space and time
l8 = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterDate('2021-01-01', '2022-01-01')
    .filterBounds(ee.Geometry.Rectangle([-124.736342, 24.521208, -66.945392, 49.382808]))
)

In [ ]:
# first step is to usually mask out poor quality observations
# this function takes a landsat image and reads the QA band 
# to determine which pixels are good or bad then
# masks (sets to nodata) poor quality pixels
def qa(img):
    # Bits 3, 4 and 5 are cloud shadow, snow and cloud, respectively.
    cloudshadow_bit_mask = 1 << 3
    snow_bit_mask = 1 << 4
    clouds_bit_mask = 1 << 5
    

    # Get the pixel QA band.
    qa = img.select('pixel_qa')

    # All flags should be set to zero, indicating clear conditions.
    cloudshadow_mask = qa.bitwiseAnd(cloudshadow_bit_mask).eq(0)
    snow_mask = qa.bitwiseAnd(snow_bit_mask).eq(0)
    cloud_mask = qa.bitwiseAnd(clouds_bit_mask).eq(0)

    # combine the masks to identify where it is clear in all cases
    mask = cloudshadow_mask.And(snow_mask).And(cloud_mask)

    # Return the masked image, scaled to reflectance, without the QA bands.
    return (
        img.updateMask(mask)
        .select("B[0-9]*")
        .copyProperties(img, ["system:time_start"])
    )

In [ ]:
# apply the QA function to mask poor quality observations
l8_qa = l8.map(qa)

In [ ]:
# apply mean reductions
l8_mean_comp = l8_qa.reduce(ee.Reducer.mean())

# apply median reduction
l8_median_comp = l8_qa.median()

# this is the equivalent calling .reduce() with 
# a median reducer
# image.reduce(ee.Reducer.median)

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(l8_median_comp, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Median Composite');
Map.addLayer(l8_mean_comp, {"bands": ["B7_mean","B5_mean","B3_mean"], "min": 0, "max": 5500, "gamma": 1.5}, 'Mean Composite');

Map.addLayerControl()


Map

### Quality band composite

Sometimes it is useful to create a composite based on a specific variable that is of interest. For example, if we are interested in creating an image of peak vegetation per-pixel we can do that.

In [ ]:
# create a function to calculate NDVI and
# add the ndvi band to the original image
def ndvi(img):
    ndvi = img.normalizedDifference(["B5","B4"]).rename("NDVI")
    return img.addBands(ndvi)

In [ ]:
# apply the ndvi function to each image
l8_ndvi = l8_qa.map(ndvi)

In [ ]:
# create a composite using the maximum ndvi value
l8_ndvi_comp = l8_ndvi.qualityMosaic("NDVI")
l8_max_comp = l8_ndvi.max()

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(l8_median_comp, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Median Composite');
Map.addLayer(l8_mean_comp, {"bands": ["B7_mean","B5_mean","B3_mean"], "min": 0, "max": 5500, "gamma": 1.5}, 'Mean Composite');
Map.addLayer(l8_ndvi_comp, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'NDVI Composite');
Map.addLayer(l8_max_comp, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Max Composite');



Map.addLayerControl()


Map

### Time based composite

While we can mosaic based on all of the imagery and a statistic, we can also composite based on data closest to a specific date of interest. To do this, we can create a band based on time and use `.quality_mosaic()`:

In [ ]:
# define a reference date that we want to composite on
REFERENCE_DATE = "2018-11-15"

# create a function that calculates the difference from an
# acquisition from the reference date
# adds the time difference as a band to the image
def add_time(img):
    t = img.date()
    t_diff = t.difference(ee.Date(REFERENCE_DATE), "day").abs().multiply(-1)
    time = ee.Image(t_diff).float().rename("time")
    time = time.updateMask(img.select([0]).mask())
    return img.addBands(time)

In [ ]:
# apply the function to calculate a time band
l8_time = l8_qa.map(add_time)

In [ ]:
# create a composite based on the maximum time band
l8_time_comp = l8_time.qualityMosaic("time")

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(l8_median_comp, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Median Composite');
Map.addLayer(l8_mean_comp, {"bands": ["B7_mean","B5_mean","B3_mean"], "min": 0, "max": 5500, "gamma": 1.5}, 'Mean Composite');
Map.addLayer(l8_time_comp, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Time Composite');

Map.addLayerControl()

Map

These are examples of compositing techniques. Again, how you composite your imagery will be based on your application. A well-known example of a peer-reviewed compositing approach is the Best Avaiable Pixel (BAP) composite ([White et al., 2014](https://www.tandfonline.com/doi/full/10.1080/07038992.2014.945827)).

It is worth noting that composite images created by reducing an image collection are able to produce pixels in any requested projection and therefore have no fixed output projection. Instead, composites have the [default projection](https://developers.google.com/earth-engine/guides/projections#the-default-projection) of WGS-84 with 1-degree resolution pixels. Composites with the default projection will be computed in whatever output projection is requested. A request occurs by displaying the composite, or by explicitly specifying a projection/scale as in an aggregation such as `reduceRegion()` or `ee.batch.Export`.

## Resampling and Reprojection

Earth Engine makes every effort to handle projection and scale so that you don't have to.  However, there are occasions where an understanding of projections is important to get the output you need.  As an example, it's time to demystify the `reproject()` calls in the previous examples.  Earth Engine requests inputs to your computations in the projection and scale of the output.  The map created using `geemap` has a [Maps Mercator projection](http://epsg.io/3857).  The scale is determined from the map's zoom level.  When you add something to this map, Earth Engine secretly reprojects the input data to Mercator, resampling (with nearest neighbor) to screen resolution pixels based on the map's zoom level, then does all the computations with the reprojected, resampled imagery.  In the previous examples, the `reproject()` calls force the computations to be done at the resolution of the input pixels.


To demonstrate the resampling done by Earth Engine, we are going to re-run the edge detection and display with and without the reprojection.


In [ ]:
# calculate edges without reprojection
edges_variable = (
    x_edges.pow(2).add(y_edges.pow(2)).sqrt()
)

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 21)

# Display gamma stretches of the input image.
Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original');
Map.addLayer(edges_variable, {"min": 0, "max": 2000,}, 'Edges with little screen pixels');
Map.addLayer(edges, {"min": 0, "max": 2000,}, 'Edges at native resolution');

Map.addLayerControl()


Map

What's happening here is that the projection specified in `reproject()` (like we used earlier) propagates backwards to the input, forcing all the computations to be performed in that projection.  If you don't specify, the computations are performed in the projection and scale of the map (Mercator) at screen resolution.


You can control how Earth Engine resamples the input with `resample()`.  By default, all resampling is done with nearest neighbor.  To change that, call `resample()` on the inputs.  Compare the input image, resampled to screen resolution with a bilinear and bicubic resampling:

In [ ]:
# Resample the image with bilinear instead of nearest neighbor.
bilinear_resampled = image.resample('bilinear');


In [ ]:
# Resample the image with bicubic instead of nearest neighbor.
bicubic_resampled = image.resample('bicubic');

In [ ]:
Map = geemap.Map()

Map.centerObject(image, 16)

# Display gamma stretches of the input image.
Map.addLayer(image, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Original (Nearest neighbor)');
Map.addLayer(bilinear_resampled, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Bilinear resampled');
Map.addLayer(bicubic_resampled, {"bands": ["B7","B5","B3"], "min": 0, "max": 5500, "gamma": 1.5}, 'Bicubic resampled');

Map.addLayerControl()


Map

Try zooming in and out, comparing to the input image resampled with nearest neighbor (i.e. without `resample()` called on it).

**_You should rarely, if ever, have to use `reproject()` and `resample()`._** Do not use `reproject()` or `resample()` unless absolutely necessary.  They are only used here for demonstration purposes.